참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.



`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)



In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import numpy.matlib as nm  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비



# 예제 05.001



## 문제에서 주어진 변수<br>Given Parameters



### 각 구간 길이<br>Length of each section



In [ ]:
L_AB_m = 0.8
L_AD_m = 2.0
L_DE_m = 1.0

L_BD_m = 2.0 - L_AB_m
L_AE_m = L_AD_m + L_DE_m



### 하중<br>Load



In [ ]:
w_AE_N_m = -6e3
P_D_N = -15e3



### 단면<br>Section



In [ ]:
b_m = 120e-3
h_m = 200e-3
h_half_m = 0.5 * h_m



## a) 최대 굽힘 응력<br>a) Maximum Bending Stress



최대 굽힘 모멘트가 발생하는 위치와 그 크기는 BMD 를 그려 보면 알 수 있다.



### A, B, C, D 지점의 x 좌표<br>x coorinates of A, B, C, D points
A 점에서 x=0 으로 한다.<br>Let x=0 at A point



In [ ]:
x_A_m = 0
x_B_m = x_A_m + L_AB_m
x_D_m = x_B_m + L_BD_m
x_E_m = x_D_m + L_DE_m



### 반력<br>Reaction Force



A점 중심의 모멘트는 분포하중, D점 하중, E점 반력이 각각 가하는 모멘트의 합이 된다.



$$
\int_{x=0}^{x=L_{AE}} wx \, dx+P_D L_{AD}+R_E L_{AE}=0\\
$$



E점 반력 $R_E$에 관련된 항만 $=$의 왼쪽에 남기고 나머지 항을 모두 오른쪽으로 옮긴다.



$$
R_E L_{AE}=-w\int_{x=0}^{x=L_{AE}} x \, dx-P_D L_{AD}\\
$$



$=$의 왼쪽과 오른쪽 모두를 $L_{AE}$ 로 나눈다.



$$
R_E=-\frac{w}{L_{AE}}\int_{x=0}^{x=L_{AE}} x \, dx-\frac{L_{AD}}{L_{AE}}P_D\\
$$



적분한다.



$$
R_E=-\frac{w}{L_{AE}}\left[\frac{1}{2}x^2\right]_{x=0}^{x=L_{AE}} 
-\frac{L_{AD}}{L_{AE}}P_D\\
$$



적분 구간을 대입한다.



$$
R_E=-\frac{wL_{AE}^2}{2L_{AE}}-\frac{L_{AD}}{L_{AE}}P_D\\
$$



분자와 분모의 $L_{AE}$를 약분한다.



$$
R_E=-\frac{wL_{AE}}{2}-\frac{L_{AD}}{L_{AE}}P_D\\
$$



E 점에서의 반력



In [ ]:
R_E_N = -0.5 * w_AE_N_m * L_AE_m - L_AD_m / L_AE_m * P_D_N
R_E_N



A 점에서의 반력
$$
R_A + w L_{AE} + P_D + R_E = 0 \\
\begin{align}
R_A &= -w L_{AE} - P_D - R_E \\
&= -w L_{AE} - P_D - \left( -\frac{wL_{AE}}{2}-\frac{L_{AD}}{L_{AE}}P_D \right) \\
&= -w L_{AE} - P_D + \left( \frac{wL_{AE}}{2}+\frac{L_{AD}}{L_{AE}}P_D \right) \\
&= -\frac{wL_{AE}}{2} - \left( \frac{L_{AE}}{L_{AE}} -\frac{L_{AD}}{L_{AE}} \right)P_D  \\
&= -\frac{wL_{AE}}{2} - \frac{L_{DE}}{L_{AE}} P_D  \\
\end{align}
$$



In [ ]:
R_A_N = - w_AE_N_m * L_AE_m - P_D_N - R_E_N
R_A_N



### A-D 구간



In [ ]:
x_AD_m_array = np.linspace(x_A_m, x_D_m, 200+1)



#### 전단력선도<br>Shear Force Diagram



A 지점에서의 반력과 분포하중을 반영한다.



$$V_{AD}=R_A+wx$$



In [ ]:
V_AD_N_array = R_A_N + w_AE_N_m * x_AD_m_array



In [ ]:
plt.fill_between(x_AD_m_array, V_AD_N_array)
plt.ylabel('V(N)')

# A 지점에서의 반력을 화살표로 표시한다.
ax = plt.gca()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlim(xmax=x_E_m)
plt.xlabel('x(m)')
plt.grid(True)
plt.show()



#### 굽힘모멘트선도<br>Bending Moment Diagram



A지점의 반력과 분포하중이 A-D 구간에 작용하는 모멘트를 반영한다.



$$M_{AD}=R_Ax+\frac{1}{2}wx^2$$



In [ ]:
M_AD_Nm_array = R_A_N * x_AD_m_array + 0.5 * w_AE_N_m * x_AD_m_array**2



In [ ]:
plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.ylabel('M(Nm)')



plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlim(xmax=x_E_m)
plt.xlabel('x(m)')
plt.grid(True)
plt.show()



### D-E 구간<br>D-E span



In [ ]:
x_DE_m_array = np.linspace(x_D_m, x_E_m, 50+1)



#### 전단력선도<br>Shear Force Diagram



A, D 지점에서의 하중, 반력과 분포 하중을 감안하면 다음과 같다.



$$ V_{DE}=R_A+P_D+w  x $$



In [ ]:
V_DE_N_array = (R_A_N + P_D_N) + w_AE_N_m * x_DE_m_array



In [ ]:
plt.fill_between(x_AD_m_array, V_AD_N_array)
plt.fill_between(x_DE_m_array, V_DE_N_array)
plt.ylabel('V(N)')

# A 지점에서의 반력을 화살표로 표시한다.
ax = plt.gca()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

# D 지점에서의 하중을 화살표로 표시한다.
ax.arrow(x_D_m, V_AD_N_array[-1], 
         0, P_D_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(P_D_N), fc='k', ec='k')

# E 지점에서의 반력을 화살표로 표시한다.
ax.arrow(x_E_m, V_DE_N_array[-1], 
         0, R_E_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_E_N), fc='k', ec='k')

plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()



#### 굽힘모멘트선도<br>Bending Moment Diagram



전단력 선도를 적분한다.



In [ ]:
M_DE_Nm_array = si.cumtrapz(V_DE_N_array, x_DE_m_array, initial=0) + M_AD_Nm_array[-1]



In [ ]:
plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.fill_between(x_DE_m_array, M_DE_Nm_array)
plt.ylabel('M(Nm)')

plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')

plt.xlabel('x(m)')

plt.grid(True)

plt.show()



### A-E 구간 전체



In [ ]:
# 전단력선도
# Shear Force Diagram

ax = plt.subplot(2, 1, 1)

plt.fill_between(x_AD_m_array, V_AD_N_array)
plt.fill_between(x_DE_m_array, V_DE_N_array)
plt.ylabel('V(N)')

ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

ax.arrow(x_D_m, V_AD_N_array[-1], 
         0, P_D_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(P_D_N), fc='k', ec='k')

ax.arrow(x_E_m, V_DE_N_array[-1], 
         0, R_E_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_E_N), fc='k', ec='k')

plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)

# 굽힘모멘트선도
# Bending Moment Diagram

plt.subplot(2, 1, 2)

plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.fill_between(x_DE_m_array, M_DE_Nm_array)
plt.ylabel('M(Nm)')

plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()



D점에서 굽힘 모멘트가 최대가 된다.



A-D 구간의 마지막 모멘트 값은 다음과 같이 알 수 있다.



In [ ]:
M_AD_Nm_array[-1]



또는 다음과 같이 모멘트의 최대값을 찾도록 시킬 수 있다.



In [ ]:
max(M_AD_Nm_array)



In [ ]:
max(M_DE_Nm_array)



In [ ]:
M_max_Nm = max(max(M_AD_Nm_array), max(M_DE_Nm_array))
M_max_Nm



최대값의 $x$ 좌표는 다음과 같이 알 수 있다.



In [ ]:
index_max_M = np.argmax(M_AD_Nm_array)
x_AD_m_array[index_max_M]



굽힘 응력과 굽힘 모멘트 사이에는 다음과 같은 관계가 있다. (p. 170, eq 5.4a)



$$\sigma_{max}=\frac{|M|_{max}c}{I}=\frac{|M|_{max}}{\frac{I}{c}}=\frac{|M|_{max}}{S}$$



단면의 모멘트 $I$, $c$ 는 다음과 같다.



$$I=\frac{bh^3}{12}\\
c=\frac{h}{2}
$$



In [ ]:
I_m4 = b_m * h_m**3 / 12
c_m = h_m * 0.5



In [ ]:
sigma_max_Pa = M_max_Nm * c_m / I_m4
sigma_max_Pa



In [ ]:
sigma_max_MPa = sigma_max_Pa * 1e-6
sigma_max_MPa



## b) 최대 굽힘 응력 단면의 응력 분포



$y$ 축 방향 범위 $\pm c$ 사이를 표시한다.



In [ ]:
y_m_array = np.linspace(-c_m, c_m, 6)
zeros_array = np.zeros_like(y_m_array)



표시할 응력값을 계산한다. 압축이면 $x$ 축 - 방향이 되도록 한다.



In [ ]:
slope = - sigma_max_Pa / c_m
sigma_Pa_array = slope * y_m_array



응력 그래프를 표시하도록 지정한다.



In [ ]:
plt.clf()

plt.plot(zeros_array, y_m_array)
plt.plot(sigma_Pa_array, y_m_array)

# 화살표를 추가한다.
for sigma_Pa, y_m in zip(sigma_Pa_array, y_m_array):
    plt.arrow(
        0.0, y_m,
        sigma_Pa*0.9, 0.0,
        head_width=1e-2, head_length=abs(sigma_Pa*.1)
)

# x, y축 이름을 지정하고 그래프를 보여준다.
plt.xlabel('$\\sigma$[Pa]')
plt.ylabel('$y$[m]')
plt.grid(True)
plt.show()



## c) 단면 B 상단 아래 25mm 지점의 응력



In [ ]:
plt.clf()

plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.fill_between(x_DE_m_array, M_DE_Nm_array)

plt.vlines(x_B_m, 0, max(M_AD_Nm_array), color='k')

plt.ylabel('M(Nm)')
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()



B 점의 x 좌표는 다음과 같다.



In [ ]:
x_B_m



A-D 구간이므로 굽힘모멘트는 다음과 같다.



$$M_{AD}=R_Ax+\frac{1}{2}wx^2$$



In [ ]:
M_B_Nm = R_A_N * x_B_m + 0.5 * w_AE_N_m * x_B_m ** 2
M_B_Nm



In [ ]:
y_B_m = c_m - 25e-3
sigma_B_Pa = -M_B_Nm * y_B_m / I_m4
sigma_B_Pa



In [ ]:
sigma_B_MPa = sigma_B_Pa * 1e-6
sigma_B_MPa



## 굽힘 응력 분포<br>Distribution of bending stress



굽힘 응력이 보의 길이 방향 `x` 위치별, 단면 안에서 중립면으로부터의 `y` 거리별로 어떻게 분포하는지 그림으로 나타내 보자.<br>
Let's visualize how bending stress is distributed across longitudinal `x` direction and vertical `y` direction.



`y` 배열을 더 높은 해상도로 새로 만든다.<br>Make a new `y` array with higher resolution.



In [ ]:
y_m_array = np.linspace(-c_m, c_m, 50 + 1)



AD 구간, DE 구간 `x` 배열을 이어 새로운 `x` 배열을 만든다.<br>Make a new `x` array by concatenating `x` arrays of A~D, D~E intervals.



In [ ]:
x_m_array = np.concatenate((x_AD_m_array, x_DE_m_array))



새로이 만들어진 `x` 배열의 크기는 다음과 같다.<br>Shape of the new `x` array.



In [ ]:
x_m_array.shape



`x` `y` 좌표 격자<br>`x` `y` coordinate grids



In [ ]:
y_m_grid, x_m_grid = np.meshgrid(y_m_array, x_m_array)



`x` `y` 좌표 격자의 크기<br>Shapes of `x` `y` coordinate grids



In [ ]:
(y_m_grid.shape, x_m_grid.shape)



모멘트 배열도 어어서 만든다.<br>Concatenate moment arrays.



In [ ]:
M_Nm_array = np.concatenate((M_AD_Nm_array, M_DE_Nm_array))



In [ ]:
M_Nm_array.shape



`x` 배열과 `M` 배열의 길이가 같은지 확인한다.



In [ ]:
assert (M_Nm_array.shape == x_m_array.shape)



모멘트 배열로 부터 모멘트 격자를 만든다.<br>Make moment grid from moment array.



In [ ]:
M_Nm_grid = nm.repmat(M_Nm_array.T, x_m_grid.shape[1], 1).T



In [ ]:
M_Nm_grid.shape



준비된 격자의 크기가 모두 같은지 확인한다.<br>Assert if all grids are all in the same shape.



In [ ]:
assert(M_Nm_grid.shape == x_m_grid.shape == y_m_grid.shape)



굽힘모멘트와 `y` 거리, 단면의 `I_m4` 값을 이용하여 굽힘 법선응력을 계산한다.<br>Calculate bending normal stress using bending moment, `y` distance, and `I_m4` of the section.



In [ ]:
sigma_Pa_grid = M_Nm_grid * y_m_grid / (-I_m4)



굽힘응력을 그림으로 표시한다.<br>Plot bending stress.



In [ ]:
# https://matplotlib.org/gallery/images_contours_and_fields/pcolormesh_levels.html
plt.pcolormesh(x_m_grid, y_m_grid, sigma_Pa_grid * 1e-6, shading="auto", cmap='jet')
plt.xlabel('x(m)')
plt.ylabel('y(m)')
# https://stackoverflow.com/questions/15908371
cbar = plt.colorbar()
cbar.set_label('$\sigma$(MPa)')
plt.show()

